In [108]:
import sys
import os
import pandas as pd
import numpy as np
sys.path.insert(0, os.path.realpath('..'))
import django
django.setup()
from firecares.firestation.models import FireDepartment, FireStation
from IPython.display import display

pd.set_option('display.max_columns', None)

# Note: you'll need to move the predictions csv into /firecares, see s3://firecares-share/predictions.2015.csv

df = pd.read_csv('../firecares/firestation/tests/mock/predictions.csv')
df

,﻿,geoid,fd_id,fd_size,region,state,lr_fire,mr_fire,lr_inj,mr_inj,lr_death,mr_death,lr_size_2,lr_size_3,mr_size_2,mr_size_3,h.fire,h.inj,h.death,h.size2,h.size3
0,1,14000US01001020100,92679,size_4,South,AL,1,7,1,0,1,NaN,0.30,0.25,0.25,0.25,NaN,NaN,NaN,NaN,NaN
1,2,14000US01001020200,92679,size_4,South,AL,2,6,2,0,1,NaN,0.60,0.25,0.00,0.00,1,1,0,0.25,0.25
2,3,14000US01001020300,92679,size_4,South,AL,3,5,3,0,1,NaN,0.25,0.60,0.50,0.50,2,1,0,0.50,0.25
3,4,14000US01001020400,92679,size_4,South,AL,4,0,NaN,0,1,NaN,0.10,0.00,0.25,0.25,3,NaN,NaN,NaN,NaN
4,5,14000US01001020500,92679,size_4,South,AL,5,0,NaN,0,1,NaN,0.20,0.00,0.10,NaN,4,1,0,0.25,0.25
5,6,14000US01001020600,92679,size_4,South,AL,6,NaN,NaN,0,1,NaN,0.40,0.75,NaN,NaN,NaN,1,1,NaN,NaN
6,7,14000US01001020700,92679,size_4,South,AL,7,0,NaN,0,1,NaN,0.50,0.80,NaN,NaN,NaN,1,NaN,NaN,NaN
7,8,14000US01001020801,NaN,NaN,South,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,14000US01001020802,NaN,NaN,South,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,14000US01001020900,NaN,NaN,South,AL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Count of departments in csv

In [111]:
cols = ['lr_fire', 'lr_size_2', 'lr_size_3', 'mr_fire', 'mr_size_2', 'mr_size_3', 'h.fire', 'h.size2', 'h.size3']

fd_id = 92679

# for xx.size2 (beyond room + beyond floor) you would want
# Sum( xx.fire * xx.size2 ) / sum( xx.fire )
# And for xx.size3 (beyond structure) you would want
# Sum( xx.fire * xx.size2 * xx.size3 ) / sum( xx. fire * xx.size2 )
df['lr_beyond_room'] = df[df['fd_id'] == fd_id].apply(lambda row: row['lr_fire'] * row['lr_size_2'], axis=1)
df['lr_beyond_structure'] = df[df['fd_id'] == fd_id].apply(lambda row: row['lr_fire'] * row['lr_size_2'] * row['lr_size_3'], axis=1)
df['mr_beyond_room'] = df[df['fd_id'] == fd_id].apply(lambda row: row['mr_fire'] * row['mr_size_2'], axis=1)
df['mr_beyond_structure'] = df[df['fd_id'] == fd_id].apply(lambda row: row['mr_fire'] * row['mr_size_2'] * row['mr_size_3'], axis=1)
df['hr_beyond_room'] = df[df['fd_id'] == fd_id].apply(lambda row: row['h.fire'] * row['h.size2'], axis=1)
df['hr_beyond_structure'] = df[df['fd_id'] == fd_id].apply(lambda row: row['h.fire'] * row['h.size2'] * row['h.size3'], axis=1)

cols2 = ['lr_beyond_room', 'lr_beyond_structure', 'mr_beyond_room', 'mr_beyond_structure', 'hr_beyond_room', 'hr_beyond_structure']

sums = df[df['fd_id'] == fd_id].groupby(['fd_id', 'state']).sum()
display(sums[cols])
display(sums[cols2])

print '% of low risk beyond room/floor fires: {}'.format(sums['lr_beyond_room'][0] / sums['lr_fire'][0])
print '% of low risk beyond structure fires: {}'.format(sums['lr_beyond_structure'][0] / sums['lr_beyond_room'][0])
print '% of medium risk beyond room/floor fires: {}'.format(sums['mr_beyond_room'][0] / sums['mr_fire'][0])
print '% of medium risk beyond structure fires: {}'.format(sums['mr_beyond_structure'][0] / sums['mr_beyond_room'][0])
print '% of high risk beyond room/floor fires: {}'.format(sums['hr_beyond_room'][0] / sums['h.fire'][0])
print '% of high risk beyond structure fires: {}'.format(sums['hr_beyond_structure'][0] / sums['hr_beyond_room'][0])

#df[df['fd_id'] == 93345]
#lr_size2 = df[['lr_fire', 'lr_size_2']].apply(lambda row: row['lr_fire'] * row['lr_size_2'])

,,lr_fire,lr_size_2,lr_size_3,mr_fire,mr_size_2,mr_size_3,h.fire,h.size2,h.size3
fd_id,state,,,,,,,,,
92679,AL,28,2.35,2.65,18,1.1,1,10,1,0.75


,,lr_beyond_room,lr_beyond_structure,mr_beyond_room,mr_beyond_structure,hr_beyond_room,hr_beyond_structure
fd_id,state,,,,,,
92679,AL,9.55,5.425,4.25,1.6875,2.25,0.5625


% of low risk beyond room/floor fires: 0.341071428571
% of low risk beyond structure fires: 0.568062827225
% of medium risk beyond room/floor fires: 0.236111111111
% of medium risk beyond structure fires: 0.397058823529
% of high risk beyond room/floor fires: 0.225
% of high risk beyond structure fires: 0.25


In [107]:
cols2 = ['lr_size_2', 'mr_size_2', 'h.size2', 'lr_size_3', 'mr_size_3', 'h.size3']

df[df['fd_id'] == 93345].groupby(['fd_id', 'state']).sum()

,,lr_fire,mr_fire,lr_inj,mr_inj,lr_death,mr_death,lr_size_2,lr_size_3,mr_size_2,mr_size_3,h.fire,h.inj,h.death,h.size2,h.size3,lr_beyond_room,lr_beyond_structure,mr_beyond_room,mr_beyond_structure,hr_beyond_room,hr_beyond_structure
fd_id,state,,,,,,,,,,,,,,,,,,,,,
93345,VA,166.284421,NaN,45.619757,4.770429,1.747018,NaN,29.837817,7.132377,NaN,NaN,177.324624,0.905018,1.279571,10.111845,7.179326,76.0249,7.836138,NaN,NaN,17.909935,2.018505


### CSV counts vs FireCARES count

In [29]:
db = len(FireDepartment.objects.count())
csv = len(df.groupby(['fd_id', 'state'])[cols].sum())
(db, csv, csv/float(db))  # hit rate

(799, 72, 0.09011264080100125)

In [95]:
np.array([1, np.nan, 2]).sum()

nan